In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

path = (r"C:\Users\gabri\Downloads\TCC\Projeto\sinresp.csv")
df = pd.read_csv(path, on_bad_lines='skip', sep=';', low_memory=False)

#Pré-processamento dos dados da base
limite = len(df) * 0.5
df = df.dropna(thresh = limite, axis = 1)
df = df.dropna(subset = ['EVOLUCAO'])

colunas_apagadas = ['SG_UF_NOT', 'ID_REGIONA', 'ID_MUNICIP', 'ID_PAIS', 'SG_UF', 'ID_RG_RESI', 'ID_MN_RESI', 'DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_INTERNA', 'DT_COLETA', 'DT_PCR', 'DT_ENCERRA', 'DT_DIGITA', 'ID_MN_INTE', 'ESTRANG', 'CO_MUN_RES', 'SG_UF_INTE']
X = df.drop(columns = colunas_apagadas)

#Covertendo colunas não numéricas para numéricas
le = LabelEncoder()
colunas_categ = ['CS_SEXO', 'ID_UNIDADE', 'CO_RG_RESI', 'CS_RACA', 'ID_RG_INTE', 'DT_EVOLUCA']
X[colunas_categ] = X[colunas_categ].apply(le.fit_transform)

X = X.apply(pd.to_numeric, errors='coerce')

#Preenchendo os valores ausentes das colunas com sua média
X = X.fillna(X.mean())
X

,SEM_NOT,SEM_PRI,CO_REGIONA,CO_MUN_NOT,ID_UNIDADE,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,CO_PAIS,CO_RG_RESI,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,VACINA,ANTIVIRAL,HOSPITAL,ID_RG_INTE,CO_RG_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,PCR_RESUL,CLASSI_FIN,CRITERIO,EVOLUCAO,DT_EVOLUCA,HISTO_VGM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,VACINA_COV,FNT_IN_COV,TRAT_COV
0,1,1,1358.000000,411390,580,2554011,0,86,3,3086.0,5,0,1.000000,1,27,2.00000,2.239459,3.030545,1.0,2.0,2.054679,1.0,1.000000,1.000000,2.000000,2.000000,1.728577,2,5.346577,2.000000,1.0,38,1358.000000,411390.0,2.0,2.0,2.00000,1.0,4.000000,2.000000,4.0,1.0,1.0,72,0,2.00000,1.000000,2.000000,2.000000,5.70828,5.000000,1.0,2,2.000000
1,2,1,1369.000000,411520,838,2586444,0,52,3,3052.0,6,0,4.000000,1,38,1.00000,2.000000,2.000000,1.0,2.0,2.000000,2.0,1.000000,2.000000,2.000000,2.000000,1.000000,1,9.000000,2.000000,1.0,70,1369.000000,411520.0,1.0,1.0,6.00000,1.0,1.000000,2.000000,4.0,1.0,1.0,180,0,2.00000,2.000000,2.000000,2.000000,2.00000,3.229481,2.0,2,3.030365
2,8,7,1354.000000,354980,547,2077396,0,1,3,3001.0,6,0,5.000000,1,23,1.00000,2.000000,2.000000,1.0,1.0,2.000000,2.0,1.000000,1.000000,2.000000,2.000000,1.000000,2,9.000000,1.000000,1.0,155,1354.000000,354980.0,1.0,2.0,4.38559,1.0,1.000000,2.000000,4.0,1.0,1.0,225,0,2.00000,2.000000,2.000000,2.000000,6.00000,4.000000,2.0,2,2.000000
3,5,5,1608.000000,430820,926,2241145,2,74,3,3074.0,6,0,1.000000,1,241,1.20309,2.000000,2.000000,1.0,1.0,2.054679,1.0,1.304715,1.404264,1.976657,1.927156,1.728577,1,1.000000,2.000000,1.0,9,1608.000000,430820.0,2.0,2.0,9.00000,1.0,1.000000,2.000000,4.0,1.0,1.0,154,0,2.10058,1.927123,2.273153,2.274199,5.70828,4.000000,1.0,2,2.000000
4,8,8,1333.000000,351880,3065,9400109,0,7,3,3007.0,6,0,9.000000,1,2,1.00000,2.000000,2.000000,2.0,1.0,2.000000,1.0,1.000000,1.000000,1.000000,2.000000,1.728577,2,9.000000,2.000000,1.0,139,1333.000000,351880.0,2.0,2.0,2.00000,2.0,1.179782,2.333538,4.0,3.0,1.0,199,0,1.00000,1.000000,2.000000,2.000000,6.00000,3.229481,1.0,2,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214000,34,34,1333.000000,353980,2302,2082411,0,47,3,3047.0,5,1,2.000000,1,2,1.00000,2.000000,2.000000,2.0,2.0,2.000000,1.0,1.000000,1.404264,1.000000,2.000000,1.728577,1,2.000000,2.000000,1.0,139,1333.000000,353980.0,2.0,3.0,2.00000,1.0,1.000000,2.000000,4.0,1.0,2.0,214,0,2.00000,1.000000,2.000000,2.000000,6.00000,2.000000,1.0,2,2.000000
214002,37,36,1462.000000,317020,789,6601804,0,71,3,3071.0,5,3,9.000000,1,108,1.00000,2.000000,2.000000,2.0,2.0,2.000000,1.0,1.000000,1.000000,2.000000,2.000000,2.000000,1,2.000000,2.000000,1.0,273,1462.000000,317020.0,1.0,2.0,4.00000,1.0,1.000000,2.000000,4.0,1.0,1.0,197,0,2.00000,1.000000,2.000000,2.000000,6.00000,5.000000,1.0,2,2.000000
214003,37,36,1336.000000,353740,2493,2083027,2,83,3,3083.0,6,3,5.059867,1,5,1.00000,2.000000,2.000000,1.0,1.0,2.000000,2.0,2.000000,1.000000,2.000000,2.000000,1.000000,1,2.000000,9.000000,1.0,141,1336.000000,353740.0,2.0,2.0,2.00000,1.0,4.000000,2.333538,5.0,1.0,1.0,125,0,2.00000,1.000000,2.000000,1.000000,6.00000,1.000000,1.0,2,9.000000
214004,36,36,1574.277836,160030,2656,2019647,0,1,3,3001.0,6,3,5.059867,1,316,1.00000,2.000000,2.000000,1.0,1.0,2.000000,1.0,1.000000,2.000000,2.000000,2.000000,2.000000,2,2.000000,2.000000,1.0,287,1571.471304,160030.0,2.0,3.0,4.38559,1.0,1.000000,1.000000,2.0,1.0,1.0,143,0,2.00000,2.000000,2.000000,2.274199,6.00000,2.000000,2.0,2,2.000000
